This notebook is for testing Convolutional Monge Mapping Normalization (hereafter, CMMN) - (put link here).

The goal is that this will improve the performance of the clf we've trained on the emotion dataset when applied to the cue dataset, even though the confusion matrix won't line up perfectly (rectangular, with slightly different classes).

An outline of how this works:
1. Load all emotion dataset, and chunk it into 30 second intervals (note - time increment doesn't matter, as long as it's consistent. But I'll stick with what they chose in their demo for now).
2. Load all cue dataset, and chunk it into 30 second intervals.
3. Fit the CMMN to the emotion dataset, then transform the cue dataset.
4. Reconstruct the cue dataset into its original form of ICs per subj.
5. Use the emotion clf on the cue dataset, take conf matrix.
5a. Additionally, take PSD and count vectors of cue before and after CMMN.

In [8]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import torch
from torch import nn

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils import check_random_state

from skorch.callbacks import EarlyStopping
from skorch.helper import predefined_split
from skorch import NeuralNetClassifier
from skorch.dataset import Dataset

from braindecode.models import SleepStagerChambon2018


import mne
from mne.datasets.sleep_physionet.age import fetch_data

from cmmn.data import load_sleep_physionet, extract_epochs
from cmmn import CMMN

# their tutorial imports above, my imports below

from scipy.io import loadmat

# emotion only missing subj 22
emotion_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']

cue_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

Load the data below. It should be fine to use absolute imports here.

Notes on the dataset - 
sampling frequency = 256 hz

The emotion dataset is natively 256 hz, and I've resampled cue to 256 hz and then done a hp filter of 1 hz to match the data preprocessing of the emotion dataset.

As the data is loaded below, ICA still needs to be calculated. They store things in slightly different structs, so the process is slightly different for each.

Also, there is a different number of channels per each - emotion has 224 and cue has 63. 

Note - this is taking too long. Save the activations after each in a new npz and then load in later piecemeal.

In [6]:
# Load the data

# emotion only missing subj 22
emotion_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']

cue_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# single_subj_emotion_test_load = loadmat('/home/austin/PycharmProjects/BOWaves/data/ds003004/after_processing/subj-01.mat')
# single_subj_cue_test_load = loadmat('/home/austin/PycharmProjects/BOWaves/data/codebooks/frolich/frolich_extract_subj_01_resampled_to_mice_hp_filtered.mat')

emotion_subj_tensors = []
cue_subj_tensors = []
cue_labels_per_subj_per_ic = []

for subj in emotion_subj_list:
    subj_data = loadmat(f'/home/austin/PycharmProjects/BOWaves/data/ds003004/after_processing/subj-{subj}.mat')
    icasphere = subj_data['icasphere']
    icaweights = subj_data['icaweights']
    data = subj_data['data']
    
    # ica
    icaact = icaweights @ icasphere @ data # resulting shape is [224, length in 256 hz]
    
    # put into tensor of shape [224, length in 256 hz]
    icaact = torch.tensor(icaact)#.unsqueeze(0)
    
    # # append to list
    # emotion_subj_tensors.append(icaact)
    
    # save to npz
    np.savez(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/emotion_subj_{subj}.npz', icaact=icaact)

for subj in cue_subj_list:
    subj_data = loadmat(f'/home/austin/PycharmProjects/BOWaves/data/codebooks/frolich/frolich_extract_subj_{subj}_resampled_to_mice_hp_filtered.mat')
    icaweights = subj_data['W']
    data = subj_data['X']
    
    icaact = icaweights @ data # resulting shape is [63, length in 256 hz]
    
    # put into tensor of shape [63, length in 256 hz]
    icaact = torch.tensor(icaact)#.unsqueeze(0)
    
    # # append to list
    # cue_subj_tensors.append(icaact)
    
    # get labels
    labels = subj_data['labels']
    
    # transform labels from ndarray of lists to list of ints
    labels = [int(label[0]) for label in labels]
    
    cue_labels_per_subj_per_ic.append(labels)
    
    # save to npz
    np.savez(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/cue_subj_{subj}.npz', icaact=icaact, labels=labels)

Reshape the data.

The end result should be a list of tensors, where each tensor is of size (K, C, T).
C is the number of channels - which should be 63 here since they've been through ICA but double check.
T is the number of time points - this will just be whatever 30 seconds is in terms of the sampling rate.
K is the number of 30 second chunks. This will be different for each subj.

For whatever remainder there is of total_time - K*T, just disregard. Over these time scales it should be no issue.

Due to memory constraints, load the ics in piecemeal, chunk them, and save the chunks again.
Do the reshaping one at a time for emotion and for cue.

Note - should I do this before or after ICA is applied? Think about it for a sec.
Each channel should still be the same time length as the original dataset. So that isn't an issue. 

But for what's termed each 'source domain' in the paper - meaning every 30 second chunk that has N channels, they take the PSD. Does it make sense to take the PSD of the individual channels? 

I think it'll be fine to take the PSD after ICA - that's what we want to map to and from. I'll double check with my coauthors after this.

In [15]:
# Reshape the data

# 30 seconds in 256 hz is 7680

emotion_chunked = [] # list of tensors of shape (K, C, T)
cue_chunked = [] # list of tensors of shape (K, C, T)

emotion_subj_tensors = []
#emotion_subj_list = ['01']#, '02', '03', '04', '05', '06', '07', '08', '09', '10']
emotion_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35']
cue_subj_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# load in emotion_subj_tensors
# for subj in emotion_subj_list:
#     subj_data = np.load(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/emotion_subj_{subj}.npz')
#     icaact = subj_data['icaact']
#     #subj = icaact
# #     emotion_subj_tensors.append(icaact)
# # 
# # for subj in emotion_subj_tensors:
# #     size = subj.shape[1]
# #     num_chunks = size // 7680
# #     #subj = subj.squeeze(0)
# #     subj = subj[:, :7680*num_chunks]
# #     subj = subj.reshape(-1, 224, 7680)
#     #emotion_chunked.append(subj)
#     num_channels, size = icaact.shape
#     num_chunks = size // 7680
#     #subj = subj.squeeze(0)
#     
#     print('shape:', icaact.shape)
#     print('num_chunks:', num_chunks)
#     icaact = icaact[:, :7680*num_chunks]
#     icaact = icaact.reshape(-1, num_channels, 7680)
#     np.savez(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/emotion_subj_{subj}_chunked.npz', icaact=icaact)
#     
for subj in cue_subj_list:
    subj_data = np.load(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/cue_subj_{subj}.npz')
    icaact = subj_data['icaact']
    labels = subj_data['labels']
    #subj = icaact
    num_channels, size = icaact.shape
    num_chunks = size // 7680
    #subj = subj.squeeze(0)
    icaact = icaact[:, :7680*num_chunks]
    icaact = icaact.reshape(-1, num_channels, 7680)
    np.savez(f'/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/cue_subj_{subj}_chunked.npz', icaact=icaact, labels=labels)

# save the chunked data as one file
#np.savez('/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/emotion_chunked.npz', *emotion_chunked) # taking a while - may fit the transform on ~10 subj.s only
    
# for subj in cue_subj_tensors:
#     size = subj.shape[2]
#     num_chunks = size // 7680
#     #subj = subj.squeeze(0)
#     subj = subj[:, :7680*num_chunks]
#     subj = subj.reshape(-1, 63, 7680)
#     cue_chunked.append(subj)

Fit the CMMN to the emotion dataset, then transform the cue dataset.

This make take a bit of time, especially considering the size of the dataset.

In [ ]:
# Fit the CMMN to the emotion dataset, then transform the cue dataset
# not enough ram on my machine, only 32 gigs. need to use Caviness.

cmmn = CMMN()
cmmn.fit(emotion_chunked)
cue_chunked_transformed = cmmn.transform(cue_chunked)

Reconstruct the cue dataset into its original form of ICs per subj.

Then save this data. 

In [ ]:
# Reconstruct the cue dataset into its original form of ICs per subj
cue_reconstructed = []

for subj in cue_chunked_transformed:
    subj = subj.reshape(63, -1)
    cue_reconstructed.append(subj)

# save the data with np.savez, including with labels
for subj in cue_reconstructed:
    labels = cue_labels_per_subj_per_ic.pop(0)
    np.savez('/home/austin/PycharmProjects/BOWaves/data/cmmn_transformed_cue_to_emotion_ics/cue_reconstructed.npz', subj=subj, labels=labels)
#np.savez('/home/austin/PycharmProjects/BOWaves/data/cue_reconstructed.npz', *cue_reconstructed)